In [1]:
import pandas as pd

In [2]:
hotspots = pd.read_csv("hot-spot.csv")
burglary = pd.read_csv("burglary_lsoa_month_2010_2025.csv")


In [3]:
filtered_burglary = burglary[burglary['LSOA code'].isin(hotspots['LSOA code'])]
filtered_burglary

,LSOA code,Year,Month_Num,Burglary_Count
0,E01000001,2011,6,1
1,E01000001,2011,8,1
2,E01000001,2011,12,2
3,E01000001,2012,1,1
4,E01000001,2012,3,2
...,...,...,...,...
79087,E01035722,2024,4,1
79088,E01035722,2024,6,3
79089,E01035722,2024,9,1
79090,E01035722,2024,10,1


In [4]:
burglary_not_in_hotspots = burglary[~burglary['LSOA code'].isin(hotspots['LSOA code'])]
unique_lsoas = burglary_not_in_hotspots['LSOA code'].unique()
unique_lsoas
len(unique_lsoas)

106

In [5]:
filtered_burglary = filtered_burglary.copy()

# Create 'date' column correctly
filtered_burglary['date'] = pd.to_datetime(
    filtered_burglary[['Year', 'Month_Num']].rename(columns={'Year': 'year', 'Month_Num': 'month'}).assign(day=1)
)

start_date = pd.to_datetime("2020-04-01")
end_date = pd.to_datetime("2021-07-31")

# Split the data
within_period = filtered_burglary[(filtered_burglary['date'] >= start_date) & (filtered_burglary['date'] <= end_date)]
outside_period = filtered_burglary[~((filtered_burglary['date'] >= start_date) & (filtered_burglary['date'] <= end_date))]

In [6]:
covid_sum = within_period.groupby('LSOA code')['Burglary_Count'].sum().reset_index()
covid_sum

,LSOA code,Burglary_Count
0,E01000001,7
1,E01000002,7
2,E01000005,26
3,E01000914,1
4,E01032739,105
5,E01032740,37
6,E01032767,1
7,E01033490,4


In [7]:
covid_with_info = pd.merge(
    covid_sum,
    hotspots[['LSOA code', 'Borough', 'longitude', 'latitude']],
    on='LSOA code',
    how='left'
)
covid_with_info

,LSOA code,Burglary_Count,Borough,longitude,latitude
0,E01000001,7,City of London,-0.096361,51.518019
1,E01000002,7,City of London,-0.092935,51.518746
2,E01000005,26,City of London,-0.074821,51.512525
3,E01000914,1,Camden,-0.116087,51.518229
4,E01032739,105,City of London,-0.086156,51.513265
5,E01032740,37,City of London,-0.107747,51.514169
6,E01032767,1,Tower Hamlets,-0.071380,51.512441
7,E01033490,4,Islington,-0.100298,51.521357


In [8]:
non_covid_sum = outside_period.groupby('LSOA code')['Burglary_Count'].sum().reset_index()
non_covid_sum

,LSOA code,Burglary_Count
0,E01000001,91
1,E01000002,98
2,E01000003,54
3,E01000005,326
4,E01000006,5
...,...,...
5153,E01035718,49
5154,E01035719,7
5155,E01035720,8
5156,E01035721,35


In [9]:
non_covid_with_info = pd.merge(
    non_covid_sum,
    hotspots[['LSOA code', 'Borough', 'longitude', 'latitude']],
    on='LSOA code',
    how='left'
)
non_covid_with_info

,LSOA code,Burglary_Count,Borough,longitude,latitude
0,E01000001,91,City of London,-0.096361,51.518019
1,E01000002,98,City of London,-0.092935,51.518746
2,E01000003,54,City of London,-0.095835,51.521646
3,E01000005,326,City of London,-0.074821,51.512525
4,E01000006,5,Barking and Dagenham,0.088361,51.538845
...,...,...,...,...,...
5153,E01035718,49,Westminster,-0.170091,51.507214
5154,E01035719,7,Westminster,-0.125479,51.492895
5155,E01035720,8,Westminster,-0.127706,51.490018
5156,E01035721,35,Westminster,-0.151836,51.489957


In [10]:
covid_with_info.to_csv("COVID_period.csv", index=False)
non_covid_with_info.to_csv("non_COVID_period.csv", index=False)